In [1]:
#!pip install simple_salesforce

In [2]:
import json
from sqlalchemy import create_engine
import pandas as pd
import pymysql
pymysql.install_as_MySQLdb()

In [3]:
# import sys
# sys.path.append('../../../../')
from config import sf_username, sf_password, sf_security_token
from config import remote_db_endpoint, remote_db_port
from config import remote_db_name, remote_db_user, remote_db_pwd

In [4]:
from simple_salesforce import Salesforce
sf = Salesforce(username=sf_username, password=sf_password, security_token=sf_security_token)

In [5]:
engine = create_engine(f"mysql://{remote_db_user}:{remote_db_pwd}@{remote_db_endpoint}:{remote_db_port}/{remote_db_name}")
conn = engine.connect()

In [6]:
class_participant_data_df = pd.read_sql("SELECT * FROM class participants", conn)
class_participant_data_df.head(30)

,ID_Class,ID_Course,Section,StartDate,EndDate
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [7]:
class_participant_data_df.rename(columns={
    'ID_Class':'ID_Class__c',
    'ID_Course':'ID_Course__c',
    'Section':'Section__c',
    'StartDate':'StartDate__c',
    'EndDate':'EndDate__c',    
}, inplace=True)
class_participant_data_df.head()

,ID_Class__c,ID_Course__c,Section__c,StartDate__c,EndDate__c
0,1,1,GWU-ARL-DATA-PT-09-0,2020-03-16,2020-03-14
1,2,1,GWDC201805DATA3,2018-05-15,2018-11-08
2,3,2,GWARL201905WEB3,2019-05-14,2019-11-07
3,4,3,GWARL201905UIUX3,2019-05-14,2019-11-07


In [8]:
class_participant_data_df = class_participant_data_df[['Section__c', 'StarDate__c', 'EndDate__c',]]
class_participant_data_df.head()

KeyError: "['StarDate__c'] not in index"

In [ ]:
student_data = student_data.to_dict('records')
student_data

In [ ]:
for rec in course_data_records:

    record = {
        'Course_Code__c': rec['Course_Code__c'],
        'Course_Name__c': rec['Course_Name__c'],
        'Credit_Hours__c': rec['Credit_Hours__c'],
        'Boot_Camp_Course__c': rec['Boot_Camp_Course__c'],
    }
    
    try:
        sf.Course__C.create(record)
    except Exception as e:
        print(e)
        

In [ ]:
course_lookup_list = []

# The `Name` column in the primary key in Salesforce objects
data = sf.query_all_iter("SELECT Course_Code__c, Name FROM Course__c")
for row in data:
    rec = {
        'Course__c': row['Name'],
        'Course_Code__c': row['Course_Code__c']
    }
    course_lookup_list.append(rec)

In [ ]:
course_lookup_df = pd.DataFrame(course_lookup_list)
course_lookup_df

In [ ]:
query = '''
    SELECT
        c.*
        ,co.CourseCode
    FROM
        class c
        INNER JOIN course co
        ON c.ID_Course = co.ID_Course
'''
class_data_df = pd.read_sql(query, conn)
class_data_df.head()

In [ ]:
class_data_df.rename(columns={
    'Section':'Section__c',
    'StartDate':'Start_Date__c',
    'EndDate':'End_Date__c',
    'CourseCode':'Course_Code__c'
}, inplace=True)

class_data_df

In [ ]:
class_data_df = pd.merge(class_data_df, course_lookup_df, how='left')
#class_data_df.drop(columns = ['ID_Class','ID_Course','CourseName','CreditHours','BootCampCourse','Course_Code__c'], inplace=True)

class_data_df.head()

In [ ]:
class_data_df
class_data_df['Start_Date__c'] = pd.to_datetime(class_data_df['Start_Date__c']).dt.date
class_data_df['End_Date__c'] = pd.to_datetime(class_data_df['End_Date__c']).dt.date

class_data_df.head()

In [ ]:
class_data_records = class_data_df.to_dict(orient='records')
class_data_records

In [ ]:
for rec in class_data_records:
 
    record = {
        'Course__c': rec['Course__c'],
        'Section__c': rec['Section__c'],
        'Start_Date__c': str(rec['Start_Date__c']),
        'End_Date__c': str(rec['End_Date__c']),
    }
    
    try:
        sf.Class__C.create(record)
    except Exception as e:
        print(e)

In [ ]:
try:
    sf.bulk.Lead.insert(student_load)
except Exception as e:
    print(e)